<table>
<tr>
<td width=15%><img src="./img/UGA.png"></img></td>
<td><center><h1>Introduction à la Recherche Opérationelle</h1><br/>M2 Statistique Science des Données (SSD)</center></td>
<td width=15%>
<a href="https://www-ljk.imag.fr/membres/Anatoli.Iouditski/" style="font-size: 16px; font-weight: bold">Anatoli Juditsky</a><br/>
<a href="http://www.iutzeler.org" style="font-size: 16px; font-weight: bold">Franck Iutzeler</a><br/>
Version Py2019 </td>
</tr>
</table>

<br/><br/>
<center><a style="font-size: 30pt; font-weight: bold">TP 2 - Dantzig Selector </a></center>
<br/><br/>

# 1. Problème de régression parcimonieuse et Dantzig Selector

On considère le modèle de régression normale
$$ y=X\theta+\sigma\xi,\;\;\xi\sim \mathcal{N}(0, I_m),$$
où  $X\in \mathbb{R}^{m\times n}$ et $y\in \mathbb{R}^m$ sont les observables, et $\theta\in \mathbb{R}^n$ est le paramètre inconnu. L'estimateur de $\theta$ de *``Dantzig Selector''* (cf. Candes, E., Tao, T. (2007). *The Dantzig selector: Statistical estimation when $p$ is much larger than $n$*. The Annals of Statistics, 2313-2351) peut être utilisé pour estimer $\theta$ dans le cas d'un modèle surparamétré, quand la dimension $n$ de $\theta$ est supérieure a la dimension de l'observation $y$. Dans ce cas l'estimateur ${\theta}_{DS}$  s'écrit comme une solution du probleme d'optimisation


$${\theta}_{DS}\in \arg\min_{\theta\in  \mathbb{R}^n} \left\{\|\theta\|_1,\;\mbox{sous contrainte}\;\|X^T(X\theta-y)\|_\infty\leq \kappa\sigma\right\},$$
où $\kappa>0$ est un *hyper-paramètre*. La valeur de $\kappa$, préconisée dans la literature, est $c q_{\mathcal{N}}\left(1-{\alpha\over 2m}\right)$, où $\alpha\in (0,1)$ est le niveau de risque choisi (e.g., $\alpha=.05$, etc) et $q_\mathcal{N}(p)$ est la $p$-quantile de la normale standardisée, et $c=\max_j\|[X]_j\|_2=\max_j\sqrt{[X]_j^T[X]_j}$ est la norme maximale de colonne de la matrice $X$.

Votre objectif dans cet exercice sera d'implementer l'estimateur ${\theta}_{DS}$ en utilisant `CVXPY`.

> **Question:** Vérifier que le problème et les contraintes peuvent se formuler via des [fonctions disponibles](hhttps://www.cvxpy.org/examples/index.html) pour CVXPY.

# 2. Un exemple

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
# Exemple jouet
n = 5
m = 3
sigma = 0.1

X = np.random.randn(m,n)
theta_true = np.array([1,0,0,0,5])
xi = np.random.randn(m)
y = np.dot(X,theta_true) + sigma*xi



In [3]:
X

array([[ 0.6809556 ,  1.39993158, -1.07573885, -0.21487585,  1.47556202],
       [-0.51569691,  0.07519816, -0.2520318 ,  0.68520616, -0.32832952],
       [ 0.80268822, -1.38307929, -0.91975332,  0.0038322 ,  1.66714571]])

In [ ]:
y

In [ ]:
theta_true

> **Question:** Trouver l'estimateur $\theta_{DS}$ à partir de $X$ et $y$ par résolution du problème d'optimisation via CVXPY avec $\kappa$ fixé à 0.2 et utiliser le solver ``ECOS``.

In [4]:
kappa = 0.2
theta = cp.Variable(n)
mini = cp.Minimize(cp.norm(theta,1))

contrainte1 = cp.norm(X.T*(X*theta-y),'inf')<= kappa*sigma
prob1 = cp.Problem(mini, constraints = [contrainte1])
prob1.solve(verbose=True, solver="ECOS")
prob1.status
theta.value


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -2.000e-02  +1e+02  4e-01  9e-01  1e+00  5e+00    ---    ---    1  1  - |  -  - 
 1  +4.674e+00  +5.235e+00  +4e+01  1e-01  2e-01  8e-01  2e+00  0.7542  1e-01   0  0  0 |  0  0
 2  +4.742e+00  +4.863e+00  +7e+00  1e-02  3e-02  2e-01  3e-01  0.8821  8e-02   0  0  0 |  0  0
 3  +5.176e+00  +5.210e+00  +2e+00  4e-03  9e-03  5e-02  9e-02  0.7527  4e-02   0  0  0 |  0  0
 4  +5.265e+00  +5.271e+00  +5e-01  1e-03  2e-03  9e-03  3e-02  0.7475  3e-02   0  0  0 |  0  0
 5  +5.287e+00  +5.287e+00  +4e-02  5e-05  1e-04  2e-04  2e-03  0.9393  2e-02   1  0  0 |  0  0
 6  +5.290e+00  +5.290e+00  +2e-03  2e-06  7e-06  1e-05  1e-04  0.9608  1e-02   1  0  0 |  0  0
 7  +5.290e+00  +5.290e+00  +2e-05  3e-08  7e-08  1e-07  1e-06  0.9890  1e-04   1  0  0 |  0  0
 8  +5.290e+00  +5.290e+00  +3e-07  3e-10  8e-

array([ 9.81579047e-12,  2.95210504e-11, -1.17121332e-01,  6.02675988e-01,
        4.57049277e+00])

# 3. Fonction "Dantzig Selector"



> **Question:** Écrivez une fonction `DSelect` qui fait appel a `CVXPY` pour calculer l'estimation ${\theta}_{DS}$. 

Cette fonction doit sortir un tuple avec les elements
* `coef`, vecteur des coefficients de regression
* `resid`, vecteur $y-X{\theta}_{DS}$ de résidus
* `status`, le statut de sortie du solver

L'appel à cette fonction devra être:

`DSelect(X, y, sigma = 1, c = 1, verb = False)`

où
* `X` et `y` sont les observables
* `sigma` est une estimation de $\sigma$
* `c` est le paramètre réel tel que la valeur de $\kappa$ dans {DS} est $ \kappa=c\,q_{\mathcal{N}}\left(1-{\alpha\over 2m}\right).$

In [10]:
import scipy.stats

def DSelect(X, y, sigma = 1, c = 1, verb = False):
    

    m,n = X.shape
     
    alpha = 0.05
    kappa = c*scipy.stats.norm.ppf(1-alpha/m)
    
    theta = cp.Variable(n)
    mini = cp.Minimize(cp.norm(theta,1))

    contrainte = cp.norm(X.T*(X*theta-y),'inf')<= kappa*sigma
    prob = cp.Problem(mini, constraints = [contrainte])
    prob.solve(verbose=True, solver="ECOS")
    
    theta_ds=theta.value
    sol_status=prob.status
    residual=y-X.dot(theta_ds)

    #### TODO
    
    return theta_ds,residual,sol_status

> **Question:** Testez votre fonction sur les deux exemples ci-dessous

### a.  Test quand on connait le *vrai* theta

In [8]:
n = 5
m = 5
X = np.random.randn(m,n)
theta_true = np.random.randn(n)
sigma = 0.005

y = np.dot(X,theta_true) + sigma*np.random.randn(m)

In [11]:
f = DSelect(X, y, sigma=sigma)
f


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.163e-02  +5e+01  5e-01  7e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  +1.199e+00  +1.837e+00  +2e+01  1e-01  2e-01  9e-01  9e-01  0.7527  2e-01   0  0  0 |  0  0
 2  +1.987e+00  +2.268e+00  +6e+00  5e-02  6e-02  4e-01  3e-01  0.7322  9e-02   0  0  0 |  0  0
 3  +2.335e+00  +2.363e+00  +5e-01  4e-03  5e-03  4e-02  3e-02  0.9172  8e-03   0  0  0 |  0  0
 4  +2.384e+00  +2.394e+00  +2e-01  2e-03  2e-03  1e-02  1e-02  0.6323  2e-01   1  0  0 |  0  0
 5  +2.402e+00  +2.404e+00  +7e-02  5e-04  5e-04  3e-03  4e-03  0.8395  1e-01   0  0  0 |  0  0
 6  +2.410e+00  +2.410e+00  +2e-03  1e-05  1e-05  2e-05  1e-04  0.9780  5e-03   1  0  0 |  0  0
 7  +2.411e+00  +2.411e+00  +2e-05  1e-07  2e-07  3e-07  1e-06  0.9890  1e-04   1  0  0 |  0  0
 8  +2.411e+00  +2.411e+00  +3e-07  1e-09  2e-

(array([-0.58143911, -1.41228946,  0.05502248,  0.16102626, -0.20073526]),
 array([-0.02610947, -0.00706644,  0.00093776, -0.00592091, -0.01188519]),
 'optimal')

In [12]:
print("True \t Dantzig selector")
for i in range(len(theta_true)):
    print('{:6.3f} \t {:6.3f}'.format(theta_true[i],f[0][i]))

True 	 Dantzig selector
-0.576 	 -0.581
-1.424 	 -1.412
 0.095 	  0.055
 0.181 	  0.161
-0.164 	 -0.201


### b. Example du papier de Candes/Tao

In [13]:
import random

n = 256
m = 72
S = 8

S_set = random.sample(range(n),k=S)

X = np.random.randn(m,n)

beta = np.zeros(n)
beta[S_set] = np.random.randn(S)

sigma = 1/3.0*np.sqrt(S/m)
y = np.dot(X,beta) + sigma*np.random.randn(m)

In [14]:
f = DSelect(X, y, sigma= sigma)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -3.552e-01  +2e+03  3e-01  1e+00  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  +6.541e+00  +9.128e+00  +2e+03  2e-01  6e-01  3e+00  2e+00  0.6051  6e-01   0  0  0 |  0  0
 2  +2.465e+00  +5.552e+00  +9e+02  5e-02  2e-01  3e+00  8e-01  0.7165  3e-01   1  0  0 |  0  0
 3  +4.875e+00  +6.174e+00  +4e+02  2e-02  7e-02  1e+00  4e-01  0.5568  2e-01   1  0  0 |  0  0
 4  +8.005e+00  +8.125e+00  +1e+02  2e-03  1e-02  1e-01  9e-02  0.8602  8e-02   1  1  0 |  0  0
 5  +9.737e+00  +9.778e+00  +5e+01  9e-04  5e-03  5e-02  5e-02  0.6250  2e-01   1  0  0 |  0  0
 6  +1.042e+01  +1.043e+01  +1e+01  2e-04  1e-03  1e-02  1e-02  0.7516  4e-02   1  0  0 |  0  0
 7  +1.052e+01  +1.052e+01  +6e+00  1e-04  5e-04  4e-03  6e-03  0.6542  2e-01   1  0  0 |  0  0
 8  +1.057e+01  +1.057e+01  +3e+00  5e-05  2e-

In [15]:
S_set

[149, 244, 33, 166, 189, 221, 217, 152]

In [16]:
import matplotlib.pyplot as plt

plt.plot(beta,'ro')
plt.plot(f[0],'b*')